## Project: Implementation of Random Forest Classifier Algorithm from Scratch

### Overview:

Developed a random forest classifier from the ground up without relying on external machine learning libraries. The goal was to build a robust ensemble learning model and gain a deeper understanding of the algorithm.

### Implementation:

- Created DecisionTree class to construct single decision trees using information gain criteria for node splitting. In addition, methods for predicting, and fitting were included.

- BootstrapSampling function randomly draws samples with replacement to create training data for individual trees. 

- RandomForest class initializes hyperparameters, and stores fitted (trained) trees in a list.

- Fit method grows each tree on a bootstrapped sample to completion using the DecisionTree class. 

- Predict aggregates predictions from all trees and returns the majority vote via a plurality calculation.


### Evaluation:  

- Tested on breast cancer dataset from scikit-learn with 10-fold cross-validation.

- Achieved well above 90% accuracy demonstrating the efficacy of the random forest approach.

- Parameters tuning of max_depth and n_estimators (number of trees) were utilized to optimize performance.

### Skills: 
Algorithm design, NumPy, Classification, Ensemble methods, Scikit-learn datasets. 

#### Outcome: 
Successful implementation of Random Forest from scratch establishing the core understanding of the algorithm. The thorough evaluation showed the viability of the approach for real-world problems. 

On the whole, this project provides an example of my skills in algorithm design and implementation, classification modeling, and hands-on experience with standard ML techniques and datasets. It is indeed worth mentioning that the from-scratch approach strengthened conceptual understanding.

### Importing the libraries

In [7]:
import numpy as np
from collections import Counter

In [25]:
def bootstrap_sample(X, y):
    N = X.shape[0]
    idxs = np.random.choice(N, N, replace=True)
    return X[idxs], y[idxs]

In [26]:
def most_common_label(y):
    return Counter(y).most_common(1)[0][0]

In [27]:
def entropy(y):
    hist = np.bincount(y)
    Ps = hist / len(y)
    return -np.sum([p*np.log2(p) for p in Ps if p>0]) 

## Creating The Architecture of Node

In [28]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

## Creating The Architecture of Decision Tree

In [29]:
class DecisionTree:
    def __init__(self, min_sample_split=2, max_depth=100, n_features=None):
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _grow_tree(self, X, y, depth=0):
        N, n_features = X.shape
        n_labels = len(np.unique(y))

        # checking the stopping criteria:
        if(depth >= self.max_depth
          or N < self.min_sample_split
          or n_labels == 1):
            leaf_value = self._most_common(y)
            return Node(value = leaf_value)

        feature_idxs = np.random.choice(n_features, self.n_features, replace=False)
        
        best_feature, best_threshold = self._best_criteria(X, y, feature_idxs)
        
        left_idxs, right_idxs = self._split(X[:, best_feature], y, best_threshold)
        
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)

        return Node(best_feature, best_threshold, left, right)

    def _best_criteria(self, X, y, feature_idxs):
        split_feature, split_threshold = None, None
        best_gain = -1
        for idx in feature_idxs:
            X_column = X[:, idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_feature = idx
                    split_threshold = threshold
        return split_feature, split_threshold

    def _information_gain(self, y, X_column, threshold):
        entropy_parent = entropy(y)

        left_idxs, right_idxs = self._split(X_column, y, threshold)

        entropy_child = (len(left_idxs) / len(y))*(entropy(y[left_idxs])) + (len(right_idxs) / len(y))*(entropy(y[right_idxs]))

        return entropy_parent - entropy_child

    def _split(self, X_column, y, threshold):
        left = np.argwhere(X_column <= threshold).flatten()
        right = np.argwhere(X_column > threshold).flatten()
        return left, right
                    
    def _most_common(self, y):
        most_common = Counter(y).most_common(1)
        return most_common[0][0]

## Creating The Structure of Random Forest Class

In [30]:
class RandomForest:
    def __init__(self, n_trees=10, min_sample_split=2, max_depth=100, n_features=None):
        self.n_trees = n_trees
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(min_sample_split=self.min_sample_split, max_depth=self.max_depth)

            X_sample, y_sample = bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        trees_preds = [tree.predict(X) for tree in self.trees]
        trees_preds = np.swapaxes(trees_preds, 0, 1)
        y_pred = [most_common_label(trees_pred) for trees_pred in trees_preds]
        return y_pred

### Importing the dataset

In [31]:
from sklearn import datasets

In [32]:
data = datasets.load_breast_cancer()
X = data.data
y = data.target

### Splitting the dataset into the Training set and Test set

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

### Training the Classifier

In [35]:
clf = RandomForest(n_trees=3, max_depth=10)

In [36]:
clf.fit(X_train, y_train)

### Predicting the Test set results & Evaluation

In [37]:
y_pred = clf.predict(X_test)

In [38]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [39]:
acc = accuracy(y_test, y_pred)

In [40]:
print("Accuracy:", acc)

Accuracy: 0.9035087719298246
